## To-do list
- metrics functions
- check for directories structure (model for prediction, databases, saving directory for the drawdown/VaR/etc plots, backup directory for model, backup directory for plots)
- import metrics (VaR, cVaR, drawdown, etc.) functions from metrics module
- Copy keys.py explanation from Touring
- Two main fund strategies: (1) passive - trade on 1st of each month, keep in basked the 10 crytpos with the most market cap; (2) active - trade every week based on predicted returns for the next week
- Let the user define how many assets to invest into
- Tem que usar sentiment analysis de twitter, reddit, google news, etc. Trabalhar com dados em T+0 e em T-1, T-2, T-3...
- Faz defasagem para RSI também
- Ver se tem algum dado de taxa de juros americana/europeia/chinesa(?)/japonesa diária
- Ou algum dado 'mundial' para ajudar na intepretação dos dados
- Também trabalhar com defasagens (1, 2, 3...) nesses dados
- When using Markovwitz, use pypfopt package
- Ao invés do clustering normal tenta filtrar por rsi > X, usa tipo as 10 maiores coins. Para isso de repente vale a pena utilizar 20 coins no total (por enquanto tá com as 15 mais líquidas) 


In [5]:
#------ Import basic packages
#import matplotlib.pyplot as plt
#import seaborn as sns
#import smtplib  # Needed for the e-mail reports
#import binance.enums  # Responsible for trading

import numpy as np
import pandas as pd
#import pandas_ta
from functions import *
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.8f' % x)

In [ ]:
#from metrics import mvcriterion, optim_mvcrit

#wallet = binance_wallet()

# fetch_data() checks for the most liquid cryptos and fetches the data from Binance
data = fetch_data(days=365*2, interval='1d')
#data.drop('cluster_num', axis=1, inplace=True)

# Let's calculate the technical indicators
data = estimate_indicators(data, rsi=20, bbands=20, roll=13)

# After the indicators are calculated, perform the normalization of the values
print('Normalizing the "close" values...')
normalized_close = data.groupby(level='asset')['close'].apply(lambda x:
                                                              np.subtract(x, data.groupby(level='asset')['close'].mean())/data.groupby(level='asset')['close'].std()
                                                              )

# Register the normalized close into the df
data['close_norm'] = np.array(normalized_close)
print('"close" values normalized and added to the dataframe.')

###########
#### THIS COMMENTED SECTION WORKS ON CLUSTERING, ON HOLD BY NOW
# Now we need to estimate the optimal number of clusters to
# allocate our resources
#clust = estimate_kmeans(data.drop('close', axis=1), cutoff=0.2)

# Sets the params for the clustering function used with apply below
#params2 = {'clusters':clust,
#           'means':1,
#           'metric':'rsi'}

# Groups the data and applies the clustering
#data = data.dropna().groupby('time', group_keys=False).apply(clustering, **params2)
###########

# Filters the specific RSI range for the passive portfolios (you may choose high or low RSI, we're going with high RSI for now)
filtered_data, dates_dict = filter_dates_and_assets(data, profile='high')

#############3 PAREI EM 1:31:39 DO VIDEO

Checking for trading pairs...
The resources directory exists, checking for trade pairs file.
Trading pairs file 'pairs_202508.csv' found, checking version.
Trading pairs file is up to date.
Loading pairs.
Pairs successfully loaded.
Requesting informations from Binance.
Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All 

/home/thiago/Documents/CryptoFund/functions.py:388: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp])


Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All done.
Requesting informations from Binance.
Request successful. Splitting data...
All done.


/home/thiago/.pyenv/versions/3.12.3/lib/python3.12/site-packages/pandas_ta/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


Calculating RSI (momentum).
Calculating Bollinger Bands (vol).
Calculating ATR (vol).
Calculating MACD (momentum).
Calculating dollar volume in millions.
Creating dollar volume moving averages.
Checking cryptos liquidity.
Creating a rank for the best cryptos in the dataset.
Estimating returns.


/home/thiago/.pyenv/versions/3.12.3/lib/python3.12/site-packages/pandas/core/indexes/base.py:681: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  result = cls(*args, **kwargs)


All indicators successfuly calculated.
Normalizing the "close" values...
"close" values normalized and added to the dataframe.


/home/thiago/.pyenv/versions/3.12.3/lib/python3.12/site-packages/pandas/core/indexes/base.py:681: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  result = cls(*args, **kwargs)
/home/thiago/Documents/CryptoFund/functions.py:652: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_temp = df_temp.reset_index().set_index(['time', 'asset']).unstack().stack()


In [6]:
def optimize_basket(prices, freq=30, l_bound=0.0, h_bound=1.0):
    """
    Description: optimize the weights of each asset for each period

    Input:  prices (pandas Series), the Series containing the assets prices;
            freq (int), the frequency on which to perform historical returns. Defaults to 30 (a month, based on daily observations);
            l_bound (float), the lower bound for each minimum asset allocation. Defaults to 0;
            h_bound (float), the higher bound for each maximum asset allocation. Defaults to 1.
        
    Outputs: --- to be seen ---
             df_temp (pandas DataFrame): filtered original df;
             dates_dict (dict): dictionary containing each asset for each day of trading.
    """
    from pypfopt import risk_models, expected_returns
    from pypfopt.efficient_frontier import EfficientFrontier

    rend = expected_returns.mean_historical_return(prices=prices, frequency=freq)

    cov_matrix = risk_models.sample_cov(prices=prices, frequency=freq)

    ef = EfficientFrontier(expected_returns=rend,
                           cov_matrix=cov_matrix,
                           weight_bounds=(l_bound, h_bound),
                           solver='SCS')
    
    weights = ef.max_sharpe()

    return ef.clean_weights()

In [7]:
filtered_data

open           high            low  \
time                asset                                                   
2023-09-21 01:00:00 BCHUSDT    218.70000000   221.00000000   211.50000000   
                    BTCUSDT  27210.25000000 27388.63000000 26800.00000000   
                    LINKUSDT     6.84600000     7.04500000     6.74100000   
                    TRXUSDT      0.08455000     0.08503000     0.08407000   
2023-09-22 01:00:00 TRXUSDT      0.08443000     0.08476000     0.08274000   
...                                     ...            ...            ...   
2025-08-20 01:00:00 ADAUSDT      0.92350000     0.95970000     0.89510000   
                    BNBUSDT    847.00000000   851.95000000   834.98000000   
                    ETHUSDT   4312.99000000  4355.00000000  4193.77000000   
                    LINKUSDT    25.79000000    25.81000000    23.89000000   
                    TRXUSDT      0.35030000     0.35260000     0.34570000   

                                      close             volume         rsi  \
time                asset                                                    
2023-09-21 01:00:00 BCHUSDT    215.40000000    133808.86900000 60.57339961   
                    BTCUSDT  27125.00000000     34207.21867000 60.41389905   
                    LINKUSDT     6.90500000   4549051.69000000 66.28917764   
                    TRXUSDT      0.08444000 182738177.59999999 73.25784642   
2023-09-22 01:00:00 TRXUSDT      0.08301000 228008891.00000000 64.98036809   
...                                     ...                ...         ...   
2025-08-20 01:00:00 ADAUSDT      0.93950000 152705691.40000001 63.71813149   
                    BNBUSDT    844.52000000     94819.92100000 63.90041623   
                    ETHUSDT   4289.42000000    282119.27260000 61.64602689   
                    LINKUSDT    24.37000000   5867796.66000000 64.13392711   
                    TRXUSDT      0.35050000 249633066.69999999 63.58823003   

                                  bb_low      bb_mid     bb_high         atr  \
time                asset                                                      
2023-09-21 01:00:00 BCHUSDT   5.20503518  5.30770831  5.41038144 -1.23780570   
                    BTCUSDT  10.13424787 10.17234924 10.21045061 -1.93274303   
                    LINKUSDT  1.89886476  1.97546763  2.05207049 -1.69178006   
                    TRXUSDT   0.07192593  0.07727685  0.08262778 -0.89076935   
2023-09-22 01:00:00 TRXUSDT   0.07246810  0.07759782  0.08272755 -0.88513919   
...                                  ...         ...         ...         ...   
2025-08-20 01:00:00 ADAUSDT   0.50199323  0.59814081  0.69428838  0.68687081   
                    BNBUSDT   6.59571552  6.68784382  6.77997213  0.73724064   
                    ETHUSDT   8.10455024  8.30859092  8.51263160  1.38910632   
                    LINKUSDT  2.73325562  3.05257846  3.37190130  1.03260565   
                    TRXUSDT   0.27626526  0.29349779  0.31073033  0.51172814   

                              ...  dollar_vol_sma14  dollar_vol_sma30  \
time                asset     ...                                       
2023-09-21 01:00:00 BCHUSDT   ...       27.75143708       24.22226689   
                    BTCUSDT   ...      791.16808947      797.10029275   
                    LINKUSDT  ...       21.25652887       20.09513931   
                    TRXUSDT   ...       18.15426033       16.97175645   
2023-09-22 01:00:00 TRXUSDT   ...       18.37866139       17.03349241   
...                           ...               ...               ...   
2025-08-20 01:00:00 ADAUSDT   ...      233.42643041      186.09058539   
                    BNBUSDT   ...      197.02187331      231.49198532   
                    ETHUSDT   ...     2947.73647605     2561.07512088   
                    LINKUSDT  ...      189.75242930      125.19442444   
                    TRXUSDT   ...      169.40629047      198.53473587   

                               

### Down here are the tweaked old functions (clustering, mostly):

In [20]:
data.drop('cluster_num', axis=1, inplace=True)
clust = estimate_kmeans(data.drop('close', axis=1), cutoff=0.2)

# Sets the params for the clustering function used with apply below
params2 = {'clusters':clust,
           'means':1,
           'metric':'rsi'}

# Groups the data and applies the clustering
data = data.dropna().groupby('time', group_keys=False).apply(clustering, **params2)

In [ ]:
# Prints the clustering graph
times = data.index.get_level_values('time').unique().to_list()[0:10]
for t in times:
    g = data.xs(t, level=1)
    plot_clusters(g)